In [11]:
#@hello world
import pandas as pd
import numpy as np

import json
from operator import itemgetter

SEPARATOR = ', '
cur_dir = "D:\\Projects\\_python\\collector-js\\"



def load_data(fname):
    s = None
    with open(cur_dir + fname) as file:
        s = file.readline()
    s = s.strip(SEPARATOR)
    json_string = "[%s]" % s
    data = json.loads(json_string)
    return data


def sort_data_by(key="time"):
    result = sorted(data, key=itemgetter(key))
    return data


def prepare_data(data):
    for d in data:
        for p in d['packages'].iteritems():
            p['upairsCount'] = len(p['uniquePairs'])

    sort_data_by()
    return data

# Structure: 
# data[index]['property:interval|time|flows|packages']['packageName:%filter_type%|packages']['packageProperty:bytesCount|packetsCount']...
# data[1]['packages']['package']['uniquePairs']

In [59]:
raw_data2 = load_data("logs_all.txt")

In [60]:
#время	am pack	am bytes	am pairs	SYN	ACK	SYN-ACK	PUSH	PUSH-ACK	TCP	ICMP	HTTP\
#1st level
data = pd.DataFrame(index=np.arange(0, len(raw_data2)), columns=("Time", "Pack", "Bytes", "Pairs", "TCP", "ICMP", "Telnet", "UDP"))
i = 0
for d in raw_data2[:3]:
    data.loc[i].Time = d['time']
    print "d"
    for _, p in d['packages'].iteritems():
        print "p"
        data.loc[i].Pairs = len(p['uniquePairs'])
        data.loc[i].Pack = p['packetsCount']
        data.loc[i].Bytes = p['bytesCount']
        for protocol_k, protocol_v in p['protocols'].iteritems():
            data.loc[i][protocol_k] = protocol_v
    i += 1

d
p
d
d
p


In [53]:
data[:3]

,Time,Pack,Bytes,Pairs,TCP,ICMP,Telnet,UDP
0,1457693200644,1999,1520061,180,0,0,0,0
1,1457693201647,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1457693202649,142,20547,30,0,0,0,0


In [51]:
data[:3]

,Time,Pack,Bytes,Pairs,TCP,ICMP,Telnet,UDP
0,1457693200644,1999,1520061,180,0,0,0,0
1,1457693202649,142,20547,30,0,0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
def resolve_dloc_path(dloc, path):
    hosts = path.split("_")
    src = hosts[0].split(":")
    dloc.SrcIP = src[0]
    
    if len(src) > 1:
        dloc.SrcPort = src[1]
    
    if len(hosts) > 1:
        dst = hosts[1].split(":")
        dloc.DstIP = dst[0]
        if len(dst) > 1:
            dloc.DstPort = dst[1]

In [64]:
# время	Host IP	host port	DST IP	DST Port	am pack	am bytes	am pairs	SYN	ACK	SYN-ACK	PUSH	PUSH-ACK	TCP	ICMP	HTTP
#2nd level
data = pd.DataFrame(index=np.arange(0, len(raw_data)), columns=("Time", "SrcIP", "SrcPort", "DstIP", "DstPort", "Pack", "Bytes", "Pairs", "TCP", "ICMP", "Telnet", "UDP"))
i = 0

for d in raw_data[:3]:
    for path, p in d['packages'].iteritems():
        data.loc[i].Time = d['time']
        resolve_dloc_path(data.loc[i], path)
        data.loc[i].Pairs = len(p['uniquePairs'])
        data.loc[i].Pack = p['packetsCount']
        data.loc[i].Bytes = p['bytesCount']
        for protocol_k, protocol_v in p['protocols'].iteritems():
            data.loc[i][protocol_k] = protocol_v
        i += 1

In [65]:
data[:10]

,Time,SrcIP,SrcPort,DstIP,DstPort,Pack,Bytes,Pairs,TCP,ICMP,Telnet,UDP
0,1457693200682,94.180.171.56,58726,NaN,NaN,1,345,1,0,0,0,1
1,1457693200682,82.136.121.9,38262,NaN,NaN,1,328,1,0,0,0,1
2,1457693200682,178.215.96.20,7185,NaN,NaN,1,345,1,0,0,0,1
3,1457693200682,74.125.163.94,80,NaN,NaN,948,1402604,1,1,0,0,0
4,1457693200682,195.154.133.84,26761,NaN,NaN,1,343,1,0,0,0,1
5,1457693200682,192.168.3.30,61143,NaN,NaN,6,406,1,1,0,0,0
6,1457693200682,192.168.4.31,65356,NaN,NaN,1,63,1,0,0,0,1
7,1457693200682,76.167.243.24,32033,NaN,NaN,1,345,1,0,0,0,1
8,1457693200682,62.205.90.223,63810,NaN,NaN,1,345,1,0,0,0,1
9,1457693200682,192.168.5.92,54909,NaN,NaN,18,2772,1,1,0,0,0


In [66]:
set(data.Time)

{nan, 1457693200682L, 1457693202683L}